In [1]:
import pandas as pd
import numpy as np

# XGBoost


On choisit XGBoost car il accepte des datasets avec des valeurs manquantes.


## Préparation des données V1 - 2023/11/13


Stratégie :  
On remplace les heures par des floats, les identifiants de station par des entiers et on supprime la colonne 'date'.  
(La suppression de la colonne 'way' a été oubliée)
On a fait une division : 90% du dataset pour le train, 10% pour le test


- Import des données


In [4]:
path = "./../../data"
x_data = pd.read_csv(path + "/Xtrain_hgcGIrA.csv", sep=",")  # features
y_data = pd.read_csv(
    path + "/Ytrain_yL5OjS4.csv", sep=",", usecols=[1]
)  # occupancy rate

NameError: name 'pd' is not defined

- Mise en forme des données


In [3]:
x_data = x_data.drop("date", axis=1)  # on supprime la colonne date
x_data["hour"] = x_data["hour"].apply(
    lambda x: int(x[:2]) if isinstance(x, str) else np.nan
)  # on transforme les strings des heures en float
# on tranforme les identifiants de gare en entiers
x_data["station"] = x_data["station"].astype("category")
cat_columns = x_data.select_dtypes(["category"]).columns
x_data[cat_columns] = x_data[cat_columns].apply(lambda x: x.cat.codes)

- Création d'un dataset de test et de validation


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    x_data.values, y_data.values, train_size=0.9
)

## XGBoostREGRESSOR - 2023/11/13 - Préparation des données V1


On utilise le regresseur et pas le classifieur car on ne cherche pas la classe de p0q0 mais sa valeur précise.


- Import de XGBRegresseur


In [5]:
from xgboost import XGBRegressor

- Test sans jouer sur les paramètres


In [6]:
XGBR = XGBRegressor()
XGBR.fit(X_train, y_train)
XGBR.score(X_test, y_test)

0.9872039534087059

Le score est assez bon, cependant, en jouant sur les paramètres, on va essayer de l'augmenter.


- Choix de learning_rate et n_estimators


In [17]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from time import time

start = time()

my_kfold = KFold(n_splits=5, shuffle=True, random_state=0)

n_estimators = [300, 500, 1000, 2000]
learning_rate = [0.005, 0.01, 0.05, 0.1]

tuned_parameters = {
    "n_estimators": n_estimators,
    "learning_rate": learning_rate,
}

XGBR_params = GridSearchCV(
    XGBRegressor(), tuned_parameters, cv=my_kfold, n_jobs=-1
)

XGBR_params.fit(X_train, y_train)
print(XGBR_params.score(X_test, y_test), XGBR_params.best_params_)
print(str(time() - start) + " sec")

0.9896284435980697 {'learning_rate': 0.05, 'n_estimators': 2000}
155.02244329452515 sec


Premier test avec

- n_estimators = [300, 500, 1000, 2000]
- learning_rate = [0.005, 0.01, 0.05, 0.1]

On obtient :

0.9896284435980697 {'learning_rate': 0.05, 'n_estimators': 2000}
155.02244329452515 sec

Le score obtenu est meilleur, mais on va essayer d'affiner ces résultats


Deuxième test avec :

- n_estimators = [1000, 1500, 2000, 2500, 3000]
- learning_rate = [0.01, 0.025, 0.05, 0.075, 0.1]

On obtient :

0.9895109476080269 {'learning_rate': 0.025, 'n_estimators': 3000}  
427.96634221076965 sec


Troisième test avec :

- n_estimators = [2750, 3000, 3250, 3500]
- learning_rate = [0.01, 0.025, 0.05, 0.075]

On obtient :

0.9895371733074723 {'learning_rate': 0.025, 'n_estimators': 3500}  
564.882648229599 sec


Quatrième test avec :

- n_estimators = [3000, 3250, 3500, 3750, 4000]
- learning_rate = [0.01, 0.025, 0.05]

On obtient :

0.9895812224115239 {'learning_rate': 0.025, 'n_estimators': 4000}
463.68704533576965 sec


On choisit de s'arrêter là pour voir l'impact des autres paramètres sur le score. Les deux meilleurs couples sont, pour n_estimators et learning_rate (0.05,2000) et (0.025,4000). On utilisera le pre;ier pour optimiser les autres paramètres, puis, on verra quel couple permet d'avoir le meilleur score final. Dans l'éventuel cas où ce serait (0.025,4000), on pourra essayer d'augmenter encore n_estimators.


- choix de max_depth et gamma


In [22]:
start = time()

my_kfold = KFold(n_splits=5, shuffle=True, random_state=0)

max_depth = [5, 6, 7, 8, 9]
min_child_weight = [5, 6, 7, 8, 9]

tuned_parameters = {
    "max_depth": max_depth,
    "min_child_weight": min_child_weight,
}

XGBR_params = GridSearchCV(
    XGBRegressor(n_estimators=2000, learning_rate=0.05),
    tuned_parameters,
    cv=my_kfold,
    n_jobs=-1,
)

XGBR_params.fit(X_train, y_train)
print(XGBR_params.score(X_test, y_test), XGBR_params.best_params_)
print(str(time() - start) + " sec")

0.9892712110007462 {'max_depth': 7, 'min_child_weight': 8}
456.5879855155945 sec


Premier test avec

- max_depth = [1, 10, 100]
- min_child_weight = [1, 10, 100]

On obtient :

0.988740513845699 {'max_depth': 10, 'min_child_weight': 10}  
442.56765842437744 sec


Deuxième test avec

- max_depth = [5, 10, 25, 50]
- min_child_weight = [5, 10, 25, 50]

On obtient :

0.989310996503391 {'max_depth': 5, 'min_child_weight': 5}  
659.9102652072906 sec


Troisième test avec

- max_depth = [3,5,7,10,15]
- min_child_weight = [3,5,7,10,15]

On obtient :

0.9892469163908968 {'max_depth': 7, 'min_child_weight': 7}
456.729660987854 sec


Quatrième test avec :

- max_depth = [5,6,7,8,9]
- min_child_weight = [5,6,7,8,9]

On obtient :

0.9892712110007462 {'max_depth': 7, 'min_child_weight': 8}  
456.5879855155945 sec


On choisit max_depth = 7 et min_child_weight = 8


- choix de reg_alpha et reg_lambda


In [27]:
start = time()

my_kfold = KFold(n_splits=5, shuffle=True, random_state=0)

reg_alpha = [0.3, 0.14, 0.15, 0.16, 0.17]
reg_lambda = [0.18, 0.19, 0.2, 0.21, 0.22]

tuned_parameters = {"reg_alpha": reg_alpha, "reg_lambda": reg_lambda}

XGBR_params = GridSearchCV(
    XGBRegressor(
        n_estimators=2000, learning_rate=0.05, max_depth=7, min_child_weight=8
    ),
    tuned_parameters,
    cv=my_kfold,
    n_jobs=-1,
)

XGBR_params.fit(X_train, y_train)
print(XGBR_params.score(X_test, y_test), XGBR_params.best_params_)
print(str(time() - start) + " sec")

0.9892045975248401 {'reg_alpha': 0.15, 'reg_lambda': 0.21}
327.5475251674652 sec


Premier test avec

- reg_alpha = [0.1, 1, 10, 100]
- reg_lambda = [0.1, 1, 10, 100]

On obtient :

0.9891920131398381 {'reg_alpha': 0.1, 'reg_lambda': 0.1}  
143.50136017799377 sec


Deuxième test avec

- reg_alpha = [0.05 , 0.1, 0.15, 0.2]
- reg_lambda = [0.05 , 0.1, 0.15, 0.2]

On obtient :

0.9891699132652103 {'reg_alpha': 0.15, 'reg_lambda': 0.2}  
241.633647441864 sec


Le résultat est inférieur à partir du 10-millième.


Troisième test avec

- reg_alpha = [0.1, 0.125, 0.15, 0.175]
- reg_lambda = [0.08 , 0.12, 0.16, 0.2, 0.25]

On obtient :

0.9891699132652103 {'reg_alpha': 0.15, 'reg_lambda': 0.2}
296.16087889671326 sec


On choisit de creuser autour de 0.15 et 0.2


Quatrième test avec

- reg_alpha = [0.3, 0.14, 0.15, 0.16, 0.17]
- reg_lambda = [0.18, 0.19, 0.2, 0.21, 0.22]

On obtient :

0.9892045975248401 {'reg_alpha': 0.15, 'reg_lambda': 0.21}  
327.5475251674652 sec


##### Test finaux


In [28]:
XGBR1 = XGBRegressor(
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=7,
    min_child_weight=8,
    reg_alpha=0.15,
    reg_lambda=0.21,
)
XGBR1.fit(X_train, y_train)
XGBR1.score(X_test, y_test)

0.9892045975248401

In [29]:
XGBR2 = XGBRegressor(
    n_estimators=4000,
    learning_rate=0.025,
    max_depth=7,
    min_child_weight=8,
    reg_alpha=0.15,
    reg_lambda=0.21,
)
XGBR2.fit(X_train, y_train)
XGBR2.score(X_test, y_test)

0.9892679703760023

On obtient bien un score meilleur que pour les valeurs de paramètres par défaut.
On explore un peu les résultats si on augment n_estimators.


In [31]:
XGBR2_1 = XGBRegressor(
    n_estimators=5000,
    learning_rate=0.025,
    max_depth=7,
    min_child_weight=8,
    reg_alpha=0.15,
    reg_lambda=0.21,
)
XGBR2_1.fit(X_train, y_train)
XGBR2_1.score(X_test, y_test)

0.9892679703760023

Le score n'évolue pas du tout, on choisit de garder XGBR2.


## XGBRegressor V2 - TODO


https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/#:~:text=In%20XGBoost%2C%20a%20hyperparameter%20is,tree%20depth%2C%20and%20regularization%20parameters.
